------------------------------------------
<a id='top'></a>
## Session contents
### [7. Map, ApplyMap, and Apply](#applying)
### [8. Groupby](#aggregating)
### [9. Merge/Join](#merging)
### [Exercise set 3](#)

In [31]:
import pandas as pd

In [ ]:
pd.merge()

----------------------
<a id='applying'></a>
## 7. Map, ApplyMap, and Apply

We've seen that pandas objects come with simple aggregation methods, and that numpy functions allow numerical operations on pandas objects. You might wonder whether we can use functions from other packages, or apply our own user-defined functions, in a similar way.

Pandas provides this functionality through the following methods:

    srs.map(f) - apply a function f element-wise to a Series (or DataFrame column)
    df.applymap(f) - apply a function f element-wise to the entire DataFrame (i.e. the DataFrame equivalent of .map)
    df.apply(f, axis) - apply a function f along columns (axis=0) or rows (axis=1) of a DataFrame
    
<img src="https://i.stack.imgur.com/DL0iQ.jpg" title="Apply" />

Many of the functions we covered in the previous sessions are shorthand for these more general methods:

    np.log(df['A']) --> df['A'].map(np.log)
    df * 2 --> df.applymap(lambda x: x*2)
    df.sum(axis=1) --> df.apply(sum, axis=1)
    
Generally, you should use map, apply, and applymap only if there is no Series or DataFrame method available. In this way, your code will be more readable and use any optimisations that pandas may have for these methods.

<div class="alert alert-block alert-info">

<span style="color:green">Additional resources</span>

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.applymap.html#pandas.DataFrame.applymap


Let's first load up a test DataFrame.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=list('ABCD'), data=np.random.randn(4, 4)*10)
df

### ApplyMap

This is probably the simplest of the apply functions to understand. Let's convert every number in df to an int (dropping the decimal).

In [ ]:
df = df.applymap(int)
df

We can create our own functions to apply to the DataFrame as well. Simple one-line functions can be declared using lambda functions. 

See http://www.secnetix.de/olli/Python/lambda_functions.hawk

In the example below, we have formatted each element of the DataFrame as a percent.

In [ ]:
df.applymap(lambda x: str(x)+'%')

More complex functions can be applied by defining a function in the usual way.

In [ ]:
def half_or_three_plus_one(x):
    """Halve if even, triple and add one if odd"""
    if x==1:
        y = 1
    elif np.mod(x, 2)==0:
        y = x / 2
    else:
        y = 3*x + 1
    return int(y)

In [ ]:
df.abs().applymap(int).applymap(half_or_three_plus_one)

### Map

This method provides the same functionality as applymap, but for Series objects. Try some of the same functions above but on a single column or row of the DataFrame only.

### Apply

The apply method is a little more difficult to understand. It applies a particular function, often an aggregation, to each row or column independently. We've encountered a few examples of an apply-like method already, e.g., the df.sum() method.

If we call .apply and use the sum function with axis=0, we will be summing up the rows (or, summing "along the columns") of our DataFrame.

In [ ]:
df.apply(sum, axis=0)

To sum along rows, simply pass axis=1 instead.

In [ ]:
df.apply(sum, axis=1)

When using .apply, the argument to your function is the row or column itself (which is of course a Series).

In [ ]:
df

In [ ]:
df.apply(lambda x: x['C'], axis=1) # x is each row of the DataFrame

In [ ]:
df.apply(lambda x: x.iloc[-1], axis=0)  # x is each column of the DataFrame

In [ ]:
def second_largest(srs):
    srs = srs.copy()  # to make a local copy of the input
    srs = srs.abs()
    srs = srs.sort_values(ascending=True)
    return srs.iloc[-2]

In [ ]:
df.apply(second_largest, axis=0)

If function that we using in the apply method returns a dict or Series, the resulting output is a DataFrame.

In [ ]:
df.apply(lambda x: pd.Series({'Median': x.median(), 'Mean': x.mean()}), axis=0)

In [ ]:
df.apply(lambda x: pd.Series({'Median': x.median(), 'Mean': x.mean()}), axis=1)

The **np.npv** function has 2 arguments, rate (a *float* which is the discount rate) and values (a *list* of future cashflows).

For example,

    np.npv(rate=0.05,values=[1,1,1,1,101])
    
will find the price of a 5 year bond with 1% annual coupons at a yield of 5%.

Run the cell below to initialise the DataFrame we'll be working with next.

In [ ]:
df_bond = pd.DataFrame({'bond_name': ['Bank_2020', 'Retailer_2018', 'JGB_3Y'], 'yield': [0.0465, 0.0573, 0.00347],
                        '2017_cashflow': [1.5, 2.5, 0.125], '2018_cashflow':[1.5, 102.5, 0.125],
                        '2019_cashflow': [1.5, 0, 100.125], '2020_cashflow':[101.5, 0, 0]})
df_bond.set_index('bond_name', drop=True, inplace=True)
df_bond

Using the **.apply, lambda functions and the np.npv function**, calculate the price of each of the three bonds above.

Hint: use a lambda function where the variable is a **row** from your DataFrame.

This exercise is fairly difficult so take a look at the solution below if you get stuck.

In [ ]:
#solutions
%load solutions/apply_sol1.py

[Back to top](#top)

----------
<a id='aggregating'></a>
# 8. GroupBy

We have now looked at applying functions and performing aggregations over an entire DataFrame. Often though, we are interested in aggregations among particular subsets of the data. For example, finding the turnover for each security, or median latency of different proccesses in our trading systems. Pandas allows this type of aggregation through the __df.groupby()__ method, which implements a "split-apply-combine" paradigm. The process is explained in the diagram below, which groups by the key and applies the sum method.

<img src="http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/03.08-split-apply-combine.png" title="Group-by explained" />

<div class="alert alert-block alert-info">

<span style="color:green">Additional resources</span>

http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb

We'll be working with volatility data, df_vols, for this section. Run the code below to import and clean the data.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_vols = pd.read_csv('data/vols.csv')
df_vols.TIMESTAMP = pd.to_datetime(df_vols.TIMESTAMP)
df_vols = df_vols.set_index('TIMESTAMP', drop=True)
df_vols.head()

Let's do some summarisation first with the .describe() method.

In [ ]:
df_vols.describe()

Suppose we want to find the mean of each column, but **per relative expiry**. We can first do a groupby on the DataFrame.

In [ ]:
g = df_vols.groupby('RELATIVE_EXPIRY')
g

The output is a DataFrameGroupBy object. Let's look at the attributes of this object with the Tab button or by running the __dir__ function on g.

You'll notice that it shares many of the same attributes and methods of the original DataFrame object. For instance, try running a few of the aggregation methods to see how they work.

In [ ]:
g.sum()

If we want to perform an aggregation over a subset of columns, we can select those columns with the dict-like syntax in the usual way. Try a few of these below.

In [ ]:
g[['FUTURE', 'VOLATILITY']].mean()

What if we wanted to group over __ranges__ of values instead? Pandas has very useful functions __pd.cut__ and __pd.qcut__ that can bin the data into value ranges and quantile ranges respectively. Let's create 10 bins around the minimum and maximum forward price.

In [ ]:
print df_vols['FUTURE'].min(), df_vols['FUTURE'].max()

In [ ]:
bins = np.linspace(210, 240, 11)
bins

In [ ]:
df_vols['FUTURE_RANGE'] = pd.cut(df_vols['FUTURE'], bins)
df_vols.head()

We can group by these to find the average ATM vol in each forward price bucket - is the result what you expected?

In [ ]:
df_vols.groupby('FUTURE_RANGE')['VOLATILITY'].mean()

Now, the above calculation is actually not very informative, because we have lumped all relative expiries. We really should group over __both__ the future prices __and__ the expiries at the same time. All we need to do is to provide a list of keys/columns to groupby.

In [ ]:
avg_vol = df_vols.groupby(['RELATIVE_EXPIRY', 'FUTURE_RANGE'])['VOLATILITY'].mean()
avg_vol

The output is a Series with a MultiIndex, where relative expiry and forward price are different levels of the index. This turns out to be a much easier way of working with data than a "3D spreadsheet" kind of structure.

Whenever we get a stacked object like above, we can call the .unstack() method to turn it back into a DataFrame.

In [ ]:
df_avg_vol = avg_vol.unstack(level='RELATIVE_EXPIRY')  # or level=0
df_avg_vol

Groupby objects also have a __.apply__ method, except the apply acts on each key's DataFrame. For instance, calculating the daily change of a few columns for each relative expiry.

In [ ]:
df_vols.groupby('RELATIVE_EXPIRY')[['FUTURE', 'VOLATILITY']].apply(lambda df: df.iloc[-1] - df.iloc[0])

[Back to top](#top)

----------------------------
<a id='merging'></a>
# 9. Merge/Join

Sometimes we will want to complement one data set with information from another data set. For example, joining a DataFrame of trades (price, size, time) with a DataFrame of instrument properties (underlying, expiry date, strike price). Pandas' main method for joining two DataFrames is __pd.merge__:

    pd.merge(df_left, df_right, on=..., how=...)
    
The 'on' argument determines which column(s) to join on. If left empty, the columns that df_left and df_right share will be used as join keys. If the columns to join on have different names between df_left and df_right, we can use the 'left_on' and 'right_on' arguments instead. To join on the index instead of a column, we use left_index=True and/or right_index=True. Alternatively, df_left.join(df_right) performs a join on the indexes.

The ‘how’ argument determines the style of join to use. Options for this argument are ‘inner’, 'outer', 'left', and 'right'. An inner join contains the intersection of the two sets of inputs. An outer join returns a join over the union of the input columns, and fills in all missing values with NaNs. The left and right joins return joins over the left and right indices respectively. Note that a right join is identical to a left join with the left/right labels swapped - so we usually just use left joins.

<img src="http://www.w3schools.com/sql/img_innerjoin.gif" title="Inner join" />
<img src="http://www.w3schools.com/sql/img_leftjoin.gif" title="Left join" />
<img src="http://www.w3schools.com/sql/img_rightjoin.gif" title="Right join" />
<img src="http://www.w3schools.com/sql/img_fulljoin.gif" title="Outer join" />

<div class="alert alert-block alert-info">

<span style="color:green">YouTube video</span>

Watch the following video until the 1 hour 13 minute mark to get a better idea of these methods.

https://www.youtube.com/watch?v=dye7rDktJ2E&start=3180

**Key methods covered:**

    pd.concat - combines two objects into a single DataFrame
    pd.merge - merges existing DataFrames

<div class="alert alert-block alert-info">

<span style="color:green">Additional resources</span>

http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.07-Merge-and-Join.ipynb

Let's load up the following data of prices and turnovers (assume a multiplier of 1).

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_turnovers = pd.DataFrame(columns=['Underlying', 'Month', 'Turnover'],
    data={'Underlying': ['HSI']*3 + ['NK225']*3 + ['HHI']*3,
          'Month': ['Jan', 'Feb', 'Mar']*3,
          'Turnover': [1000, 1100,   900,
                        300,  350,   400,
                       6000, 7000, np.nan]})
df_turnovers

In [ ]:
df_prices = pd.DataFrame(columns=['Underlying', 'Month', 'Price'],
    data={'Underlying': ['HSI']*3 + ['HHI']*3 + ['NK225']*3,
          'Month': ['Jan', 'Feb', 'Mar']*3,
          'Price': [28000, 29000, 30000,
                    11000, 12000, 115000,
                    22000, 21000, 20000]})
df_prices

Calculate the notional turnover (price times size) in local currency by joining on the appropriate key(s) with __pd.merge__.

In [ ]:
#solutions
%load solutions/merge_sol1.py

Now load the following currency data, and join them together to get the forex rates for each underlying.

In [ ]:
df_currency = pd.DataFrame({'Underlying': ['NK225', 'HSI', 'HHI'], 'Currency': ['JPY', 'HKD', 'HKD']})
df_currency

In [ ]:
df_forex = pd.DataFrame({'Currency': ['HKD', 'JPY', 'KRW'], 'Rate': [6, 80, 850]})
df_forex

In [ ]:
#solutions
%load solutions/merge_sol2.py

Finally, join currency data onto the notional turnover data and convert the notional turnover to AUD (in millions).

In [ ]:
#solutions
%load solutions/merge_sol3.py

[Back to top](#top)

## Exercise set 3 (unavailable externally)

For this exercise set we'll use our HSI options data. For now, just run the following cells to get our trade data from OneTick - we'll spend time learning how to use OneTick later. You'll need to install the following package first:
>pip install optiver.etl

In [32]:
import pandas as pd
from etl.onetick import otq, query
import datetime as dt

start = dt.datetime(2018, 3, 13, 0, 0)
end = dt.datetime(2018, 3, 13, 23, 59)

# Trade data
q = query.tick_query('Trade_Tick_Analysis', 'ATLAS_IN',
                     start, end, 'Australia/Sydney',
                     symbol_regex='ATLAS_IN::opa_in_hsi_tko_001.XHKF',
                     columns=['EEID_TIMESTAMP', 'FEEDCODE', 'TRADE_PRICE', 'TRADE_VOLUME', 'THEO_PRICE', 'DELTA'])

df_trd = otq.query(q)
df_trd = df_trd.drop(columns=['Time', 'SYMBOL_NAME'])
df_trd['EEID_TIMESTAMP'] = pd.to_datetime(df_trd['EEID_TIMESTAMP'])
df_trd = df_trd.set_index('EEID_TIMESTAMP')
df_trd = df_trd[df_trd['FEEDCODE'].str.startswith('HSI')]

# Instrument data
q = query.tick_query('Instrument', 'XHKF',
                     start, end, 'Australia/Sydney',
                     symbol_regex='XHKF::HSI',
                     columns=['FEEDCODE', 'KIND', 'STRIKE_PRICE', 'EXPIRY_DATE'])

df_ins = otq.query(q)
df_ins = df_ins.drop(columns=['Time', 'SYMBOL_NAME'])

1. Add a new column EDGE to df_trd that contains the total edge of that trade in AUD.
2. Merge the instrument data into the trade data.
3. Calculate the total edge and trade volumes per delta bucket (delta 0-10, 10-20, 20-30, etc.), expiry date, and instrument kind.
4. Unstack that dataframe so that it's easier to view.
5. Sort the dataframe in descending order of edge.
6. What was the total edge for the day?

In [33]:
list(df_trd)

['DELTA',
 'FEEDCODE',
 'THEO_PRICE',
 'THEO_PRICE_ADJUST',
 'TRADE_PRICE',
 'TRADE_VOLUME']

In [34]:
df_trd['EDGE'] = (df_trd['THEO_PRICE'] - df_trd['TRADE_PRICE']).abs() * df_trd['TRADE_VOLUME']

In [37]:
print len(df_ins)
df_ins = df_ins.drop_duplicates('FEEDCODE')
print len(df_ins)

2086
2071


In [38]:
pd.merge(df_trd, df_ins, how='left')

,DELTA,FEEDCODE,THEO_PRICE,THEO_PRICE_ADJUST,TRADE_PRICE,TRADE_VOLUME,EDGE,EXPIRY_DATE,KIND,STRIKE_PRICE
0,1.000000,HSIH8,31550.162750,0.000000,31550.0,1.0,0.162750,2018-03-27 13:00:00,Future,NaN
1,1.000000,HSIH8,31550.162750,0.000000,31550.0,3.0,0.488250,2018-03-27 13:00:00,Future,NaN
2,1.000000,HSIH8,31550.162750,0.000000,31550.0,23.0,3.743250,2018-03-27 13:00:00,Future,NaN
3,1.000000,HSIH8,31550.162750,0.000000,31550.0,1.0,0.162750,2018-03-27 13:00:00,Future,NaN
4,1.000000,HSIH8,31550.162750,0.000000,31550.0,1.0,0.162750,2018-03-27 13:00:00,Future,NaN
5,1.000000,HSIH8,31550.162750,0.000000,31550.0,4.0,0.651000,2018-03-27 13:00:00,Future,NaN
6,1.000000,HSIH8,31550.162750,0.000000,31550.0,10.0,1.627500,2018-03-27 13:00:00,Future,NaN
7,1.000000,HSIH8,31550.162750,0.000000,31550.0,7.0,1.139250,2018-03-27 13:00:00,Future,NaN
8,1.000000,HSIH8,31550.162750,0.000000,31550.0,21.0,3.417750,2018-03-27 13:00:00,Future,NaN
9,1.000000,HSIH8,31550.162750,0.000000,31550.0,7.0,1.139250,2018-03-27 13:00:00,Future,NaN


In [30]:
# 1. Calculate edge as |theo-trade|
df_trd['EDGE'] = (df_trd['THEO_PRICE'] - df_trd['TRADE_PRICE']).abs() * df_trd['TRADE_VOLUME']

# 2. Join on instrument data
df_ins = df_ins.drop_duplicates('FEEDCODE')  # drop any duplicate instrument ticks
df = pd.merge(df_trd, df_ins, 'left')  # left join so we don't drop trades

# 3. Make delta buckets, sum over edge and volume
delta_range = np.linspace(0, 1, 11)
df['DELTA_BUCKET'] = pd.cut(df['DELTA'].abs(), delta_range)

groups = df.groupby(['DELTA_BUCKET', 'EXPIRY_DATE', 'KIND'])
groups = groups[['TRADE_VOLUME', 'EDGE']]  # keep these two columns only
groups_sum = groups.sum().sort_values('EDGE', ascending=False)

# 4. Unstack
groups_unstacked = groups_sum.unstack('KIND')  # pull out from index to columns

ValueError: Cannot sort by column EDGE in a multi-index you need to explicitly provide all the levels

In [ ]:
groups_unstacked